In [9]:
from abc import ABC, abstractmethod
from typing import Dict, List, Any

class BaseExampleSelector(ABC):
    """Interface for selecting examples to include in prompts."""

    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """Select which examples to use based on the inputs."""
        
    @abstractmethod
    def add_example(self, example: Dict[str, str]) -> Any:
        """Add new example to store."""


In [10]:
examples = [
    {"input": "hi", "output": "ciao"},
    {"input": "bye", "output": "arrivederci"},
    {"input": "soccer", "output": "calcio"},
]


In [11]:
from langchain_core.example_selectors.base import BaseExampleSelector

class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples  # Store examples

    def add_example(self, example):
        """Adds a new example to the selector."""
        self.examples.append(example)

    def select_examples(self, input_variables):
        """Selects an example with the closest word length."""
        new_word = input_variables["input"]
        new_word_length = len(new_word)

        best_match = None
        smallest_diff = float("inf")

        # Find the closest matching example based on word length
        for example in self.examples:
            current_diff = abs(len(example["input"]) - new_word_length)
            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example

        return [best_match] if best_match else []


In [ ]:
example_selector = CustomExampleSelector(examples)

# Selecting an example based on input "okay"
print(example_selector.select_examples({"input": "okay"}))  
# Expected Output: [{'input': 'bye', 'output': 'arrivederci'}]


In [ ]:
example_selector.add_example({"input": "hand", "output": "mano"})

# Selecting an example for input "okay" again
print(example_selector.select_examples({"input": "okay"}))  
# Expected Output: [{'input': 'hand', 'output': 'mano'}]


In [14]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate.from_template("Input: {input} -> Output: {output}")


In [15]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} -> Output:",
    prefix="Translate the following words from English to Italian:",
    input_variables=["input"],
)


In [ ]:
print(prompt.format(input="word"))
